In [14]:
import numpy as np
import feature_engine as fe
import pandas as pd
import gc

In [2]:
train = pd.read_feather('./train_data.ftr')
catg = ['B_30', 'B_31', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
cont_features = sorted([f for f in train.columns if f not in catg + ["customed_ID", 'target', 'S_2']])

# Feature Engineering
Common steps to follow:
- ...

Here are the strategies to try for each model:
- Aggregate features for every customer
- Use rows as they are


In [143]:
def aggregate_features(data, categorical_cols, numerical_cols):
    
    categorical_cols = [c for c in categorical_cols if c not in ["customer_ID", "S_2", "target"]]
    numerical_cols = [c for c in numerical_cols if c not in ["customer_ID", "S_2", "target"]]

    
    num_agg = data.groupby("customer_ID")[numerical_cols].agg(['mean', 'std', 'min', 'max', 'last'])
    num_agg.columns = ['_'.join(x) for x in num_agg.columns]

    num_cols = num_agg.columns

    catg_agg = data.groupby('customer_ID')[categorical_cols].agg(['count', 'last', 'nunique'])
    catg_agg.columns = ['_'.join(x) for x in catg_agg.columns]

    catg_cols = catg_agg.columns

    temp = pd.concat([num_agg, catg_agg], axis=1)
    del num_agg
    del catg_agg
    gc.collect()

    return temp, list(num_cols), list(catg_cols)

In [144]:
agg_df, agg_num_cols, agg_catg_cols = aggregate_features(train, catg, cont_features)

In [145]:
null_counts = pd.DataFrame(agg_df.isna().sum(), columns=["null_count"])
drop_null_cols = null_counts[null_counts["null_count"] > (len(agg_df) * 0.7)].index
drop_null_cols

Index(['B_29_mean', 'B_29_std', 'B_29_min', 'B_29_max', 'B_29_last',
       'B_39_mean', 'B_39_std', 'B_39_min', 'B_39_max', 'B_39_last',
       ...
       'R_26_std', 'R_26_min', 'R_26_max', 'R_26_last', 'R_9_mean', 'R_9_std',
       'R_9_min', 'R_9_max', 'R_9_last', 'D_66_last'],
      dtype='object', length=117)

In [147]:
df_to_save = agg_df.drop(columns=drop_null_cols)
df_to_save = df_to_save.reset_index()
for col in drop_null_cols:
    if col in agg_catg_cols:
        agg_catg_cols.remove(col)
    elif col in agg_num_cols:
        agg_num_cols.remove(col)

800

In [149]:
print(df_to_save.select_dtypes("category").isna().sum())
print("rows left after dropping: {}".format(len(df_to_save.dropna())))

B_30_last       31
B_38_last       31
D_114_last    4739
D_116_last    4739
D_117_last    4739
D_120_last    4739
D_126_last       0
D_63_last        0
D_64_last        0
D_68_last     5251
dtype: int64
rows left after dropping: 11059


The categorical variables that contains NaN values obscures the categorical encoding. Dropping rows that contains NaN values is not an option because it almost drops all of the dataset. Therefore missing data imputation must be applied.

In [150]:
df_to_save[agg_catg_cols]

,B_30_count,B_30_last,B_30_nunique,B_31_count,B_31_last,B_31_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,...,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,13,0.0,1,13,1.0,1,13,2.0,1,13,...,CR,1,13,O,1,0,0,13,6.0,1
1,13,0.0,1,13,1.0,1,13,2.0,1,13,...,CO,1,13,O,1,0,0,13,6.0,1
2,13,0.0,1,13,1.0,1,13,1.0,1,13,...,CO,1,13,R,1,0,0,13,6.0,1
3,13,0.0,1,13,1.0,1,13,2.0,1,13,...,CO,1,13,O,1,0,0,13,3.0,3
4,13,0.0,1,13,1.0,1,13,1.0,2,13,...,CO,1,13,O,1,13,1,13,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,13,0.0,1,13,1.0,1,13,2.0,1,13,...,CO,1,13,U,1,13,1,13,6.0,1
458909,13,0.0,2,13,1.0,1,13,7.0,2,13,...,CO,1,13,R,1,0,0,13,6.0,1
458910,13,0.0,1,13,1.0,1,13,3.0,2,13,...,CO,1,13,U,1,0,0,13,5.0,1
458911,13,0.0,1,13,1.0,1,13,3.0,1,13,...,CO,1,13,U,2,0,0,13,3.0,2


In [153]:
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer

median_imputer = MeanMedianImputer(
                   imputation_method='median')

median_imputer.fit(df_to_save[agg_num_cols])
df_to_save[agg_num_cols] = median_imputer.transform(df_to_save[agg_num_cols])

In [154]:
categorical_imputer = CategoricalImputer()

categorical_imputer.fit(df_to_save[agg_catg_cols])
df_to_save[agg_catg_cols] = categorical_imputer.transform(df_to_save[agg_catg_cols])


c:\Users\asude\anaconda3\envs\ML\lib\site-packages\feature_engine\imputation\categorical.py:212: FutureWarning: The `inplace` parameter in pandas.Categorical.add_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  X[variable].cat.add_categories(
c:\Users\asude\anaconda3\envs\ML\lib\site-packages\feature_engine\imputation\categorical.py:212: FutureWarning: The `inplace` parameter in pandas.Categorical.add_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  X[variable].cat.add_categories(
c:\Users\asude\anaconda3\envs\ML\lib\site-packages\feature_engine\imputation\categorical.py:212: FutureWarning: The `inplace` parameter in pandas.Categorical.add_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  X[variable].cat.add_categories

In [155]:
from feature_engine.encoding import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(df_to_save[agg_catg_cols])
encoded_df = encoder.transform(df_to_save[agg_catg_cols])

In [156]:
temp = df_to_save.drop(columns=agg_catg_cols)
df_to_save = pd.concat([temp, encoded_df], axis=1)
df_to_save

,customer_ID,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_10_mean,B_10_std,B_10_min,B_10_max,...,D_64_last_U,D_64_last_,D_68_last_6.0,D_68_last_3.0,D_68_last_5.0,D_68_last_4.0,D_68_last_2.0,D_68_last_1.0,D_68_last_Missing,D_68_last_0.0
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.012009,0.006546,0.001930,0.021652,0.009384,0.270264,0.181835,0.096191,0.741699,...,0,0,1,0,0,0,0,0,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.025650,0.027750,0.006710,0.109619,0.034698,0.298828,0.003044,0.293945,0.302734,...,0,0,1,0,0,0,0,0,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.004387,0.002786,0.001472,0.009995,0.004284,0.273682,0.052867,0.162109,0.302734,...,0,0,1,0,0,0,0,0,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.059875,0.080538,0.005909,0.280029,0.012566,0.306641,0.079525,0.192993,0.431885,...,0,0,0,1,0,0,0,0,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.005939,0.002475,0.000776,0.009804,0.007679,0.100342,0.074579,0.044739,0.260742,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.029175,0.014286,0.006084,0.051941,0.028519,0.591309,0.373031,0.366211,1.417969,...,1,0,1,0,0,0,0,0,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.368408,0.051249,0.292480,0.452148,0.292480,0.042603,0.016051,0.021011,0.083557,...,0,0,1,0,0,0,0,0,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.043030,0.040382,0.013000,0.162476,0.020569,0.268555,0.069366,0.092590,0.302734,...,1,0,0,0,1,0,0,0,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.018158,0.009637,0.000281,0.030563,0.015839,0.039734,0.041169,0.006924,0.154907,...,1,0,0,1,0,0,0,0,0,0


In [157]:
df_to_save.isna().sum().sum()

0

In [171]:
df_to_save.dtypes.value_counts()

float16     613
float64     152
int32        47
int64        24
object        1
category      1
dtype: int64

In [168]:
df_to_save["D_126_last"] = df_to_save["D_126_last"].cat.remove_categories("Missing")

In [169]:
df_to_save.to_feather("aggregated_data.ftr")

0